In [1]:
import os
import json
import time
import sys
import pickle
import argparse
import dateutil.parser
from random import shuffle, randrange, uniform

import numpy as np
from sqlite3 import dbapi2 as sqlite3
from hashlib import md5
from flask import Flask, request, session, url_for, redirect, \
     render_template, abort, g, flash, _app_ctx_stack
from flask_limiter import Limiter
from werkzeug import check_password_hash, generate_password_hash
import pymongo

from utils import safe_pickle_dump, strip_version, isvalidid, Config
import os
import sys
import pickle
# non-standard imports
import numpy as np
from sklearn import svm
from sqlite3 import dbapi2 as sqlite3
# local imports
from utils import safe_pickle_dump, strip_version, Config

In [2]:
if not os.path.isfile(Config.database_path):
  print("the database file as.db should exist. You can create an empty database with sqlite3 as.db < schema.sql")
  sys.exit()

In [3]:
sqldb = sqlite3.connect(Config.database_path)

In [5]:
def query_db(query, args=(), one=False):
  """Queries the database and returns a list of dictionaries."""
  cur = sqldb.execute(query, args)
  rv = cur.fetchall()
  return (rv[0] if rv else None) if one else rv

In [6]:
users = query_db('''select * from user''')

In [7]:
users

[(1,
  'scbebetterme',
  'pbkdf2:sha1:1000$nuUjaWwl$7e43984cb1f14dc7b57d2fe6399551772c962615',
  1503146987)]

In [14]:
saved_ids=query_db('select * from library')

In [24]:

ids=[]
for saved_id in saved_ids:
    ids.append(saved_id[1])
    


In [27]:
idversions = [db[id]['_rawid'] +'v'+ str(db[id]['_version']) for id in ids]


In [28]:
idversions

['1708.00871v1', '1707.09589v2', '1708.05606v1']

In [22]:

article_titles=[]
for saved_id in saved_ids:
    id = saved_id[1]
    article_titles.append(db[id]['title'])


In [23]:
article_titles

['Spread of entanglement in a Sachdev-Ye-Kitaev chain',
 'Dispersive SYK model: band structure and quantum chaos',
 'Null hypersurface quantization, electromagnetic duality and asympotic\n  symmetries of Maxwell theory']

In [20]:
db[a[1]]

{'_rawid': '1708.00871',
 '_version': 1,
 'arxiv_comment': '1+46 pages, 11 figures',
 'arxiv_primary_category': {'scheme': 'http://arxiv.org/schemas/atom',
  'term': 'hep-th'},
 'author': 'Xiao-Liang Qi',
 'author_detail': {'name': 'Xiao-Liang Qi'},
 'authors': [{'name': 'Yingfei Gu'},
  {'name': 'Andrew Lucas'},
  {'name': 'Xiao-Liang Qi'}],
 'guidislink': True,
 'id': 'http://arxiv.org/abs/1708.00871v1',
 'link': 'http://arxiv.org/abs/1708.00871v1',
 'links': [{'href': 'http://arxiv.org/abs/1708.00871v1',
   'rel': 'alternate',
   'type': 'text/html'},
  {'href': 'http://arxiv.org/pdf/1708.00871v1',
   'rel': 'related',
   'title': 'pdf',
   'type': 'application/pdf'}],
 'published': '2017-08-02T18:01:57Z',
 'published_parsed': time.struct_time(tm_year=2017, tm_mon=8, tm_mday=2, tm_hour=18, tm_min=1, tm_sec=57, tm_wday=2, tm_yday=214, tm_isdst=0),
 'summary': "We study the spread of R\\'enyi entropy between two halves of a\nSachdev-Ye-Kitaev (SYK) chain of Majorana fermions, prepared

In [55]:
def encode_feedparser_dict(d):
  """ 
  helper function to get rid of feedparser bs with a deep copy. 
  I hate when libs wrap simple things in their own classes.
  """
  if isinstance(d, feedparser.FeedParserDict) or isinstance(d, dict):
    j = {}
    for k in d.keys():
      j[k] = encode_feedparser_dict(d[k])
    return j
  elif isinstance(d, list):
    l = []
    for k in d:
      l.append(encode_feedparser_dict(k))
    return l
  else:
    return d

def parse_arxiv_url(url):
  """ 
  examples is http://arxiv.org/abs/1512.08756v2
  we want to extract the raw id and the version
  """
  ix = url.rfind('/')
  idversion = url[ix+1:] # extract just the id (and the version)
  parts = idversion.split('v')
  assert len(parts) == 2, 'error parsing url ' + url
  return parts[0], int(parts[1])

In [2]:
db = pickle.load(open(Config.db_path, 'rb'))
db['1611.07917']

KeyError: '1611.07917'

In [32]:
db = pickle.load(open(Config.db_path, 'rb'))
del db['1511.04021']
del db['1412.6087']
del db['1611.07917']
safe_pickle_dump(db, Config.db_path)

In [69]:
db = pickle.load(open(Config.db_path, 'rb'))
xml='<?xml version="1.0" encoding="UTF-8"?><feed xmlns="http://www.w3.org/2005/Atom"><link href="http://arxiv.org/api/query?search_query%3Dcat%3Ahep-th%26id_list%3D%26start%3D20%26max_results%3D1" rel="self" type="application/atom+xml"/><title type="html">ArXiv Query: search_query=cat:hep-th&amp;id_list=&amp;start=20&amp;max_results=1</title><id>http://arxiv.org/api/w7OBLXfq/5JLmD3NcNop0iAVzqE</id><updated>2017-08-20T00:00:00-04:00</updated><opensearch:totalResults xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">118402</opensearch:totalResults><opensearch:startIndex xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">20</opensearch:startIndex><opensearch:itemsPerPage xmlns:opensearch="http://a9.com/-/spec/opensearch/1.1/">1</opensearch:itemsPerPage><entry><id>http://arxiv.org/abs/1604.07818v1</id><updated>2016-04-26T18:00:09Z</updated><published>2016-04-26T18:00:09Z</published><title>\nComments on the Sachdev-Ye-Kitaev model</title><summary> We study a quantum mechanical model proposed by Sachdev, Ye and Kitaev. The\nmodel consists of $N$ Majorana fermions with random interactions of a few\nfermions at a time. It it tractable in the large $N$ limit, where the classical\nvariable is a bilocal fermion bilinear. The model becomes strongly interacting\nat low energies where it develops an emergent conformal symmetry. We study two\nand four point functions of the fundamental fermions. This provides the\nspectrum of physical excitations for the bilocal field.\n</summary><author><name>Juan Maldacena</name></author><author><name>Douglas Stanford</name></author><arxiv:primary_category xmlns:arxiv="http://arxiv.org/schemas/atom" term="hep-th" scheme="http://arxiv.org/schemas/atom"/><link href="http://arxiv.org/abs/1604.07818v1 " rel="alternate" type="text/html"/><link title="pdf" href="http://arxiv.org/pdf/1604.07818v1 " rel="related" type="application/pdf"/></entry></feed>'
parse = feedparser.parse(xml)
num_added=0
for e in parse.entries:
    j = encode_feedparser_dict(e)
    rawid, version = parse_arxiv_url(j['id'])
    j['_rawid'] = rawid
    j['_version'] = version
    j['tags']=[{'label': None, 'term': 'hep-th', 'scheme': 'http://arxiv.org/schemas/atom'}]
    if not rawid in db or j['_version'] > db[rawid]['_version']:
        db[rawid] = j
        num_added+=1
        print('Updated %s added %s' % (j['updated'].encode('utf-8'), j['title'].encode('utf-8')))

if num_added>0:
    print('Saving database with %d papers to %s' % (len(db), Config.db_path))
    safe_pickle_dump(db, Config.db_path)


Updated b'2016-04-26T18:00:09Z' added b'Comments on the Sachdev-Ye-Kitaev model'
Saving database with 1031 papers to db.p


In [18]:
db = pickle.load(open(Config.db_path, 'rb'))


In [29]:
db = pickle.load(open(Config.db_path, 'rb'))
db['1703.00365']

{'_rawid': '1703.00365',
 '_version': 1,
 'arxiv_primary_category': {'scheme': 'http://arxiv.org/schemas/atom',
  'term': 'cond-mat.stat-mech'},
 'author': 'Frank Verstraete',
 'author_detail': {'name': 'Frank Verstraete'},
 'authors': [{'name': 'Matthias Bal'},
  {'name': 'Michaël Mariën'},
  {'name': 'Jutho Haegeman'},
  {'name': 'Frank Verstraete'}],
 'guidislink': True,
 'id': 'http://arxiv.org/abs/1703.00365v1',
 'link': 'http://arxiv.org/abs/1703.00365v1',
 'links': [{'href': 'http://arxiv.org/abs/1703.00365v1',
   'rel': 'alternate',
   'type': 'text/html'},
  {'href': 'http://arxiv.org/pdf/1703.00365v1',
   'rel': 'related',
   'title': 'pdf',
   'type': 'application/pdf'}],
 'published': '2017-03-01T18:00:09Z',
 'published_parsed': time.struct_time(tm_year=2017, tm_mon=3, tm_mday=1, tm_hour=18, tm_min=0, tm_sec=9, tm_wday=2, tm_yday=60, tm_isdst=0),
 'summary': 'A renormalization group flow of Hamiltonians for two-dimensional classical\npartition functions is constructed using

In [22]:
'9605187' in db.keys()

True

In [20]:
safe_pickle_dump(db, Config.db_path)

In [28]:
db['1008.4964']['link']

'http://arxiv.org/abs/1008.4964v3'

In [34]:
db = pickle.load(open(Config.db_path, 'rb'))
for pid,j in db.items():
    try:
        j['tags']
    except:
        print(pid,"\n")
        
safe_pickle_dump(db, Config.db_path)